In [11]:
#import necessary libraries
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import cv2
import numpy as np

In [12]:
class ImageUploader:
    def __init__(self, root):
        self.root = root
        self.root.title("Image Stitching App")

        self.img1_path = None
        self.img2_path = None

        # Create frames for layout
        self.top_frame = tk.Frame(root)
        self.top_frame.pack(side=tk.TOP, pady=10)

        self.middle_frame = tk.Frame(root)
        self.middle_frame.pack(side=tk.TOP, pady=10)

        self.bottom_frame = tk.Frame(root)
        self.bottom_frame.pack(side=tk.TOP, pady=10)

        # Create and place widgets in the top frame
        self.label1 = tk.Label(self.top_frame, text="Upload first image:")
        self.label1.grid(row=0, column=0, padx=5)

        self.button1 = tk.Button(self.top_frame, text="Browse", command=self.upload_image1)
        self.button1.grid(row=0, column=1, padx=5)

        self.canvas1 = tk.Canvas(self.top_frame, width=300, height=300)
        self.canvas1.grid(row=1, column=0, columnspan=2, pady=10)

        self.label2 = tk.Label(self.top_frame, text="Upload second image:")
        self.label2.grid(row=0, column=2, padx=5)

        self.button2 = tk.Button(self.top_frame, text="Browse", command=self.upload_image2)
        self.button2.grid(row=0, column=3, padx=5)

        self.canvas2 = tk.Canvas(self.top_frame, width=300, height=300)
        self.canvas2.grid(row=1, column=2, columnspan=2, pady=10)

        # Create and place stitch button in the middle frame
        self.button3 = tk.Button(self.middle_frame, text="Stitch Images", command=self.stitch_images, width=20)
        self.button3.pack(pady=10)

        # Create canvas for the stitched image in the bottom frame
        self.stitched_canvas = tk.Canvas(self.bottom_frame, width=600, height=300)
        self.stitched_canvas.pack()

    def upload_image1(self):
        self.img1_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.tiff")])
        if self.img1_path:
            img = Image.open(self.img1_path)
            img.thumbnail((300, 300))
            img = ImageTk.PhotoImage(img)
            self.canvas1.create_image(150, 150, image=img)
            self.canvas1.image = img

    def upload_image2(self):
        self.img2_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg;*.jpeg;*.png;*.bmp;*.tiff")])
        if self.img2_path:
            img = Image.open(self.img2_path)
            img.thumbnail((300, 300))
            img = ImageTk.PhotoImage(img)
            self.canvas2.create_image(150, 150, image=img)
            self.canvas2.image = img

    def stitch_images(self):
        if self.img1_path and self.img2_path:
            try:
                # Load images using OpenCV
                image1 = cv2.imread(self.img1_path)
                image2 = cv2.imread(self.img2_path)

                # Convert images to RGB using OpenCV
                image1_RGB = cv2.cvtColor(image1, cv2.COLOR_BGR2RGB)
                image2_RGB = cv2.cvtColor(image2, cv2.COLOR_BGR2RGB)

                # Convert images to grayscale using OpenCV
                image1_gray = cv2.cvtColor(image1, cv2.COLOR_BGR2GRAY)
                image2_gray = cv2.cvtColor(image2, cv2.COLOR_BGR2GRAY)

                # Initialize ORB detector
                orb = cv2.ORB_create()

                # Detect keypoints and descriptors in the images
                keypoints1, descriptors1 = orb.detectAndCompute(image1_gray, None)
                keypoints2, descriptors2 = orb.detectAndCompute(image2_gray, None)

                # Create a BFMatcher object
                bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

                # Match descriptors of the two images
                matches = bf.match(descriptors1, descriptors2)

                # Sort the matches based on distance and get the best matches, we can adjust the number of best matches
                matches = sorted(matches, key=lambda x: x.distance)
                good_matches = matches[:70]

                if len(good_matches) > 4:
                     
                     # Extract location of good matches
                     src_pts = np.float32([keypoints2[m.trainIdx].pt for m in good_matches]).reshape(-1, 1, 2)
                     dst_pts = np.float32([keypoints1[m.queryIdx].pt for m in good_matches]).reshape(-1, 1, 2)

                     # Find the transformation matrix using RANSAC
                     M, mask = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 5.0)
                     
                     # Get the size of image2
                     h, w, _ = image2_RGB.shape
                     
                     # Use the warpPerspective function to warp image2 to image1
                     warpImg = cv2.warpPerspective(image2_RGB, M, (image1_RGB.shape[1] + image2_RGB.shape[1], image1_RGB.shape[0]))
                     
                     # Add image1 to the left side of warpImg
                     warpImg[0:image1_RGB.shape[0], 0:image1_RGB.shape[1]] = image1_RGB
                     
                     # Convert the stitched image to a format suitable for Tkinter
                     img = Image.fromarray(warpImg)
                     img.thumbnail((600, 300))
                     img = ImageTk.PhotoImage(img)
                     
                     # Update the canvas with the stitched image
                     self.stitched_canvas.create_image(300, 150, image=img)
                     self.stitched_canvas.image = img
                else:

                    print("Not enough matches are found - {}/{}".format(len(matches), 4))



            except Exception as e:
                messagebox.showerror("Error", f"Error during stitching: {str(e)}")

        else:
            messagebox.showerror("Error", "Please upload two images first.")

root = tk.Tk()
app = ImageUploader(root)
root.mainloop()
